# Sympy code for FDM expansion of LBM diffusive equation up to Sixth-order

In [1]:
import sympy as sp
# -----------------------------------------------------------------------------
# 1) Symbols (parameters and step sizes)
# -----------------------------------------------------------------------------
dx, dt = sp.symbols('Delta_x Delta_t', positive=True, real=True)
s1, s2, w0, nu = sp.symbols('s_{1} s_{2} w_{0} \\nu', real=True)

alpha1 = sp.simplify(1 - s1/2 - w0*s2/2)
alpha2 = sp.simplify((w0-1)*s2 + 1)
beta1  = sp.simplify(w0*s1*s2/2 - s1*s2/2 - w0*s2/2 + s1/2 +s2 -1)
beta2  = sp.simplify(-w0*s1*s2 + w0*s2 + s1 - 1)
gamma  = sp.simplify((s1-1)*(s2-1))
# -----------------------------------------------------------------------------
# 2) Continuous variables and field
# -----------------------------------------------------------------------------
x, t = sp.symbols('x t', real=True)
phi = sp.Function('phi')
# Shorthand for the base (continuous) field value at (x,t)
PHI = phi(x, t)

In [3]:
def taylor_phi(sx=0, st=0, order_x=6, order_t=6):
    expr = sp.S(0)
    # Keep all mixed terms up to the specified individual orders.
    for m in range(order_x + 1):
        for n in range(order_t + 1):
            # skip the (0,0) term? no, keep it.
            term = ( (sx*dx)**m * (st*dt)**n
                    / (sp.factorial(m)*sp.factorial(n))
                    * sp.diff(PHI, x, m, t, n) )
            expr += term
    return sp.expand(expr)

In [4]:
# -----------------------------------------------------------------------------
# 4) Discrete stencil terms (expanded)
# -----------------------------------------------------------------------------
# phi_x^{t+1}, phi_x^{t}, phi_x^{t-1}, phi_x^{t-2}
phi_t_p1 = taylor_phi(sx=0,  st=+1, order_x=0, order_t=6)
display(phi_t_p1)
phi_t_0  = taylor_phi(sx=0,  st=0,  order_x=0, order_t=0)  # exactly phi(x,t)
display(phi_t_0)
phi_t_m1 = taylor_phi(sx=0,  st=-1, order_x=0, order_t=6)
display(phi_t_m1)
phi_t_m2 = taylor_phi(sx=0,  st=-2, order_x=0, order_t=6)
display(phi_t_m2)

# phi_{x±1}^{t}
phi_xm1_t0 = taylor_phi(sx=-1, st=0,  order_x=6, order_t=0)
display(phi_xm1_t0)
phi_xp1_t0 = taylor_phi(sx=+1, st=0,  order_x=6, order_t=0)
display(phi_xp1_t0)

# phi_{x±1}^{t-1}
phi_xm1_tm1 = taylor_phi(sx=-1, st=-1, order_x=6, order_t=6)
phi_xp1_tm1 = taylor_phi(sx=+1, st=-1, order_x=6, order_t=6)

Delta_t**6*Derivative(phi(x, t), (t, 6))/720 + Delta_t**5*Derivative(phi(x, t), (t, 5))/120 + Delta_t**4*Derivative(phi(x, t), (t, 4))/24 + Delta_t**3*Derivative(phi(x, t), (t, 3))/6 + Delta_t**2*Derivative(phi(x, t), (t, 2))/2 + Delta_t*Derivative(phi(x, t), t) + phi(x, t)

phi(x, t)

Delta_t**6*Derivative(phi(x, t), (t, 6))/720 - Delta_t**5*Derivative(phi(x, t), (t, 5))/120 + Delta_t**4*Derivative(phi(x, t), (t, 4))/24 - Delta_t**3*Derivative(phi(x, t), (t, 3))/6 + Delta_t**2*Derivative(phi(x, t), (t, 2))/2 - Delta_t*Derivative(phi(x, t), t) + phi(x, t)

4*Delta_t**6*Derivative(phi(x, t), (t, 6))/45 - 4*Delta_t**5*Derivative(phi(x, t), (t, 5))/15 + 2*Delta_t**4*Derivative(phi(x, t), (t, 4))/3 - 4*Delta_t**3*Derivative(phi(x, t), (t, 3))/3 + 2*Delta_t**2*Derivative(phi(x, t), (t, 2)) - 2*Delta_t*Derivative(phi(x, t), t) + phi(x, t)

Delta_x**6*Derivative(phi(x, t), (x, 6))/720 - Delta_x**5*Derivative(phi(x, t), (x, 5))/120 + Delta_x**4*Derivative(phi(x, t), (x, 4))/24 - Delta_x**3*Derivative(phi(x, t), (x, 3))/6 + Delta_x**2*Derivative(phi(x, t), (x, 2))/2 - Delta_x*Derivative(phi(x, t), x) + phi(x, t)

Delta_x**6*Derivative(phi(x, t), (x, 6))/720 + Delta_x**5*Derivative(phi(x, t), (x, 5))/120 + Delta_x**4*Derivative(phi(x, t), (x, 4))/24 + Delta_x**3*Derivative(phi(x, t), (x, 3))/6 + Delta_x**2*Derivative(phi(x, t), (x, 2))/2 + Delta_x*Derivative(phi(x, t), x) + phi(x, t)

In [5]:
# -----------------------------------------------------------------------------
# 5) Build the discrete equation (expanded) and move everything to one side
# -----------------------------------------------------------------------------
lhs = phi_t_p1

rhs = (alpha1*phi_xm1_t0 + alpha2*phi_t_0 + alpha1*phi_xp1_t0
       + beta1*phi_xm1_tm1 + beta2*phi_t_m1 + beta1*phi_xp1_tm1
       + gamma * phi_t_m2)

residual = sp.simplify(sp.expand(lhs - rhs))  # = 0 is the modified equation

In [6]:
max_dx=6
max_dt=3
max_total=5
expr = sp.expand(residual)
poly = sp.Poly(expr, dx, dt, domain='EX')  # keep symbolic coeffs
kept = sp.S(0)

for (px, pt), coeff in poly.terms():
    if (px <= max_dx) and (pt <= max_dt) and ((px + pt) <= max_total):
        kept += coeff * dx**px * dt**pt

In [8]:
display(sp.simplify(kept.subs(dt**2*dx**2,0)))

Delta_t**3*(7*s_{1}*s_{2} - 6*s_{1} - 6*s_{2} + 6)*Derivative(phi(x, t), (t, 3))/6 + Delta_t**2*(-3*s_{1}*s_{2} + 2*s_{1} + 2*s_{2})*Derivative(phi(x, t), (t, 2))/2 + Delta_t*Delta_x**4*(s_{1}*s_{2}*w_{0} - s_{1}*s_{2} + s_{1} - s_{2}*w_{0} + 2*s_{2} - 2)*Derivative(phi(x, t), t, (x, 4))/24 + Delta_t*Delta_x**2*(s_{1}*s_{2}*w_{0} - s_{1}*s_{2} + s_{1} - s_{2}*w_{0} + 2*s_{2} - 2)*Derivative(phi(x, t), t, (x, 2))/2 + Delta_t*s_{1}*s_{2}*Derivative(phi(x, t), t) + Delta_x**4*s_{2}*(-s_{1}*w_{0} + s_{1} + 2*w_{0} - 2)*Derivative(phi(x, t), (x, 4))/24 + Delta_x**2*s_{2}*(-s_{1}*w_{0} + s_{1} + 2*w_{0} - 2)*Derivative(phi(x, t), (x, 2))/2

In [48]:
rel1=(kept.subs(dt**2*dx**2, 0)
                 .subs(PHI.diff(t, 3), ((1-w0)*((2-s1)/(2*s1)))**2*dx**4/dt**2*nu*PHI.diff(x, 6))
                 .subs(PHI.diff(t, 2), (1-w0)*((2-s1)/(2*s1))*dx*dx/dt*nu*PHI.diff(x, 4))
                 .subs(PHI.diff(x, 4, t), nu*PHI.diff(x, 6))
                 .subs(PHI.diff(t, x, 2), nu*PHI.diff(x, 4)) )
rel1

Delta_t**3*(7*Delta_x**4*\nu*s_{2}*(1 - w_{0})**2*(2 - s_{1})**2*Derivative(phi(x, t), (x, 6))/(24*Delta_t**2*s_{1}) - Delta_x**4*\nu*(1 - w_{0})**2*(2 - s_{1})**2*Derivative(phi(x, t), (x, 6))/(4*Delta_t**2*s_{1}) - Delta_x**4*\nu*s_{2}*(1 - w_{0})**2*(2 - s_{1})**2*Derivative(phi(x, t), (x, 6))/(4*Delta_t**2*s_{1}**2) + Delta_x**4*\nu*(1 - w_{0})**2*(2 - s_{1})**2*Derivative(phi(x, t), (x, 6))/(4*Delta_t**2*s_{1}**2)) + Delta_t**2*(-3*Delta_x**2*\nu*s_{2}*(1 - w_{0})*(2 - s_{1})*Derivative(phi(x, t), (x, 4))/(4*Delta_t) + Delta_x**2*\nu*(1 - w_{0})*(2 - s_{1})*Derivative(phi(x, t), (x, 4))/(2*Delta_t) + Delta_x**2*\nu*s_{2}*(1 - w_{0})*(2 - s_{1})*Derivative(phi(x, t), (x, 4))/(2*Delta_t*s_{1})) + Delta_t*Delta_x**4*(\nu*s_{1}*s_{2}*w_{0}*Derivative(phi(x, t), (x, 6))/24 - \nu*s_{1}*s_{2}*Derivative(phi(x, t), (x, 6))/24 + \nu*s_{1}*Derivative(phi(x, t), (x, 6))/24 - \nu*s_{2}*w_{0}*Derivative(phi(x, t), (x, 6))/24 + \nu*s_{2}*Derivative(phi(x, t), (x, 6))/12 - \nu*Derivative(phi(x, 

In [49]:
rel2=(sp.collect(sp.expand(rel1),[PHI.diff(x, 6),PHI.diff(x, 4),PHI.diff(x, 2)]) )
rel2

Delta_t*s_{1}*s_{2}*Derivative(phi(x, t), t) + (-Delta_x**2*s_{1}*s_{2}*w_{0}/2 + Delta_x**2*s_{1}*s_{2}/2 + Delta_x**2*s_{2}*w_{0} - Delta_x**2*s_{2})*Derivative(phi(x, t), (x, 2)) + (-Delta_t*Delta_x**2*\nu*s_{1}*s_{2}*w_{0}/4 + Delta_t*Delta_x**2*\nu*s_{1}*s_{2}/4 + Delta_t*Delta_x**2*\nu*s_{1}*w_{0}/2 + 3*Delta_t*Delta_x**2*\nu*s_{2}*w_{0}/2 - Delta_t*Delta_x**2*\nu*s_{2} - Delta_t*Delta_x**2*\nu*w_{0} - Delta_t*Delta_x**2*\nu*s_{2}*w_{0}/s_{1} + Delta_t*Delta_x**2*\nu*s_{2}/s_{1} - Delta_x**4*s_{1}*s_{2}*w_{0}/24 + Delta_x**4*s_{1}*s_{2}/24 + Delta_x**4*s_{2}*w_{0}/12 - Delta_x**4*s_{2}/12)*Derivative(phi(x, t), (x, 4)) + (7*Delta_t*Delta_x**4*\nu*s_{1}*s_{2}*w_{0}**2/24 - 13*Delta_t*Delta_x**4*\nu*s_{1}*s_{2}*w_{0}/24 + Delta_t*Delta_x**4*\nu*s_{1}*s_{2}/4 - Delta_t*Delta_x**4*\nu*s_{1}*w_{0}**2/4 + Delta_t*Delta_x**4*\nu*s_{1}*w_{0}/2 - 5*Delta_t*Delta_x**4*\nu*s_{1}/24 - 17*Delta_t*Delta_x**4*\nu*s_{2}*w_{0}**2/12 + 67*Delta_t*Delta_x**4*\nu*s_{2}*w_{0}/24 - 4*Delta_t*Delta_x**

In [50]:
rel3=(sp.collect(sp.expand(rel2/(dt*s1*s2)),[PHI.diff(x, 6),PHI.diff(x, 4),PHI.diff(x, 2)])
             .subs(sp.expand(dx*dx*(-w0/2+sp.Rational(1,2)+w0/s1-1/s1)/dt),sp.factor(dx*dx*(-w0/2+sp.Rational(1,2)+w0/s1-1/s1)/dt))
             .subs(sp.factor(-dx*dx*(-w0/2+sp.Rational(1,2)+w0/s1-1/s1)/dt), nu)
             .subs(sp.expand(dx*dx*dx*dx*(-w0/2+sp.Rational(1,2)+w0/s1-1/s1)/(12*dt)), -dx*dx*nu/12)
             .subs(sp.expand(dx*dx*nu*(-w0/4 + sp.Rational(1,6) + w0/(2*s2) + 3*w0/(2*s1) - 1/s1 - w0/(s1*s2) - w0/s1**2 + 1/s1**2 ) ), sp.factor(sp.expand(dx*dx*nu*(-w0/4 + sp.Rational(1,6) + w0/(2*s2) + 3*w0/(2*s1) - 1/s1 - w0/(s1*s2) - w0/s1**2 + 1/s1**2 ) ))))
rel3

-Delta_x**2*\nu*(3*s_{1}**2*s_{2}*w_{0} - 2*s_{1}**2*s_{2} - 6*s_{1}**2*w_{0} - 18*s_{1}*s_{2}*w_{0} + 12*s_{1}*s_{2} + 12*s_{1}*w_{0} + 12*s_{2}*w_{0} - 12*s_{2})*Derivative(phi(x, t), (x, 4))/(12*s_{1}**2*s_{2}) - \nu*Derivative(phi(x, t), (x, 2)) + (7*Delta_x**4*\nu*w_{0}**2/24 - 13*Delta_x**4*\nu*w_{0}/24 + Delta_x**4*\nu/4 - Delta_x**4*\nu*w_{0}**2/(4*s_{2}) + Delta_x**4*\nu*w_{0}/(2*s_{2}) - 5*Delta_x**4*\nu/(24*s_{2}) - 17*Delta_x**4*\nu*w_{0}**2/(12*s_{1}) + 67*Delta_x**4*\nu*w_{0}/(24*s_{1}) - 4*Delta_x**4*\nu/(3*s_{1}) + 5*Delta_x**4*\nu*w_{0}**2/(4*s_{1}*s_{2}) - 5*Delta_x**4*\nu*w_{0}/(2*s_{1}*s_{2}) + 7*Delta_x**4*\nu/(6*s_{1}*s_{2}) + 13*Delta_x**4*\nu*w_{0}**2/(6*s_{1}**2) - 13*Delta_x**4*\nu*w_{0}/(3*s_{1}**2) + 13*Delta_x**4*\nu/(6*s_{1}**2) - 2*Delta_x**4*\nu*w_{0}**2/(s_{1}**2*s_{2}) + 4*Delta_x**4*\nu*w_{0}/(s_{1}**2*s_{2}) - 2*Delta_x**4*\nu/(s_{1}**2*s_{2}) - Delta_x**4*\nu*w_{0}**2/s_{1}**3 + 2*Delta_x**4*\nu*w_{0}/s_{1}**3 - Delta_x**4*\nu/s_{1}**3 + Delta_x**4*

## Sixth-Order Term

In [67]:
sixrelex=sp.expand(dx**4*nu*(7*w0**2/24 - 13*w0/24 + sp.Rational(1,4) - w0**2/(4*s2) + w0/(2*s2) - 5/(24*s2) - 17*w0**2/(12*s1) + 67*w0/(24*s1) - 4/(3*s1) + 5*w0**2/(4*s1*s2) 
                    - 5*w0/(2*s1*s2) + 7/(6*s1*s2) + 13*w0**2/(6*s1**2) - 13*w0/(3*s1**2) + 13/(6*s1**2) - 2*w0**2/(s1**2*s2) + 4*w0/(s1**2*s2) 
                    - 2/(s1**2*s2) - w0**2/(s1**3) + 2*w0/(s1**3) - 1/(s1**3) + w0**2/(s1**3*s2) - 2*w0/(s1**3*s2) + 1/(s1**3*s2) ) )
sixrelex

7*Delta_x**4*\nu*w_{0}**2/24 - 13*Delta_x**4*\nu*w_{0}/24 + Delta_x**4*\nu/4 - Delta_x**4*\nu*w_{0}**2/(4*s_{2}) + Delta_x**4*\nu*w_{0}/(2*s_{2}) - 5*Delta_x**4*\nu/(24*s_{2}) - 17*Delta_x**4*\nu*w_{0}**2/(12*s_{1}) + 67*Delta_x**4*\nu*w_{0}/(24*s_{1}) - 4*Delta_x**4*\nu/(3*s_{1}) + 5*Delta_x**4*\nu*w_{0}**2/(4*s_{1}*s_{2}) - 5*Delta_x**4*\nu*w_{0}/(2*s_{1}*s_{2}) + 7*Delta_x**4*\nu/(6*s_{1}*s_{2}) + 13*Delta_x**4*\nu*w_{0}**2/(6*s_{1}**2) - 13*Delta_x**4*\nu*w_{0}/(3*s_{1}**2) + 13*Delta_x**4*\nu/(6*s_{1}**2) - 2*Delta_x**4*\nu*w_{0}**2/(s_{1}**2*s_{2}) + 4*Delta_x**4*\nu*w_{0}/(s_{1}**2*s_{2}) - 2*Delta_x**4*\nu/(s_{1}**2*s_{2}) - Delta_x**4*\nu*w_{0}**2/s_{1}**3 + 2*Delta_x**4*\nu*w_{0}/s_{1}**3 - Delta_x**4*\nu/s_{1}**3 + Delta_x**4*\nu*w_{0}**2/(s_{1}**3*s_{2}) - 2*Delta_x**4*\nu*w_{0}/(s_{1}**3*s_{2}) + Delta_x**4*\nu/(s_{1}**3*s_{2})

In [68]:
sixrels1=sp.factor(sp.expand(dx**4*nu*(7*w0**2/24 - 13*w0/24 + sp.Rational(1,4) - w0**2/(4*s2) + w0/(2*s2) - 5/(24*s2) - 17*w0**2/(12*s1) + 67*w0/(24*s1) - 4/(3*s1) + 5*w0**2/(4*s1*s2) 
                    - 5*w0/(2*s1*s2) + 7/(6*s1*s2) + 13*w0**2/(6*s1**2) - 13*w0/(3*s1**2) + 13/(6*s1**2) - 2*w0**2/(s1**2*s2) + 4*w0/(s1**2*s2) 
                    - 2/(s1**2*s2) - w0**2/(s1**3) + 2*w0/(s1**3) - 1/(s1**3) + w0**2/(s1**3*s2) - 2*w0/(s1**3*s2) + 1/(s1**3*s2) ) ))
sixrel=sixrels1*24*s1**3*s2/(dx**4*nu)
sixrel

7*s_{1}**3*s_{2}*w_{0}**2 - 13*s_{1}**3*s_{2}*w_{0} + 6*s_{1}**3*s_{2} - 6*s_{1}**3*w_{0}**2 + 12*s_{1}**3*w_{0} - 5*s_{1}**3 - 34*s_{1}**2*s_{2}*w_{0}**2 + 67*s_{1}**2*s_{2}*w_{0} - 32*s_{1}**2*s_{2} + 30*s_{1}**2*w_{0}**2 - 60*s_{1}**2*w_{0} + 28*s_{1}**2 + 52*s_{1}*s_{2}*w_{0}**2 - 104*s_{1}*s_{2}*w_{0} + 52*s_{1}*s_{2} - 48*s_{1}*w_{0}**2 + 96*s_{1}*w_{0} - 48*s_{1} - 24*s_{2}*w_{0}**2 + 48*s_{2}*w_{0} - 24*s_{2} + 24*w_{0}**2 - 48*w_{0} + 24

In [70]:
rel3s2=rel3.subs(sixrelex,sixrels1)
rel3s2

Delta_x**4*\nu*(7*s_{1}**3*s_{2}*w_{0}**2 - 13*s_{1}**3*s_{2}*w_{0} + 6*s_{1}**3*s_{2} - 6*s_{1}**3*w_{0}**2 + 12*s_{1}**3*w_{0} - 5*s_{1}**3 - 34*s_{1}**2*s_{2}*w_{0}**2 + 67*s_{1}**2*s_{2}*w_{0} - 32*s_{1}**2*s_{2} + 30*s_{1}**2*w_{0}**2 - 60*s_{1}**2*w_{0} + 28*s_{1}**2 + 52*s_{1}*s_{2}*w_{0}**2 - 104*s_{1}*s_{2}*w_{0} + 52*s_{1}*s_{2} - 48*s_{1}*w_{0}**2 + 96*s_{1}*w_{0} - 48*s_{1} - 24*s_{2}*w_{0}**2 + 48*s_{2}*w_{0} - 24*s_{2} + 24*w_{0}**2 - 48*w_{0} + 24)*Derivative(phi(x, t), (x, 6))/(24*s_{1}**3*s_{2}) - Delta_x**2*\nu*(3*s_{1}**2*s_{2}*w_{0} - 2*s_{1}**2*s_{2} - 6*s_{1}**2*w_{0} - 18*s_{1}*s_{2}*w_{0} + 12*s_{1}*s_{2} + 12*s_{1}*w_{0} + 12*s_{2}*w_{0} - 12*s_{2})*Derivative(phi(x, t), (x, 4))/(12*s_{1}**2*s_{2}) - \nu*Derivative(phi(x, t), (x, 2)) + Derivative(phi(x, t), t)

In [64]:
sp.collect(sixrel.subs(w0,sp.Rational(2,3)),s2)

s_{1}**3/3 + 4*s_{1}**2/3 - 16*s_{1}/3 + s_{2}*(4*s_{1}**3/9 - 22*s_{1}**2/9 + 52*s_{1}/9 - 8/3) + 8/3

In [76]:
fac1=sp.factor(s1**3/3 + 4*s1**2/3 - 16*s1/3 + sp.Rational(8,3))
display(fac1)
fac2=sp.factor(4*s1**3/9 - 22*s1**2/9 + 52*s1/9 - sp.Rational(8,3))
display(fac2)

(s_{1} - 2)*(s_{1}**2 + 6*s_{1} - 4)/3

2*(2*s_{1}**3 - 11*s_{1}**2 + 26*s_{1} - 12)/9

## Fouth-Order Term

In [11]:
sp.expand(dx*dx*nu*(-w0/4 + sp.Rational(1,6) + w0/(2*s2) + 3*w0/(2*s1) - 1/s1 - w0/(s1*s2) - w0/s1**2 + 1/s1**2 ) )

-Delta_x**2*\nu*w_{0}/4 + Delta_x**2*\nu/6 + Delta_x**2*\nu*w_{0}/(2*s_{2}) + 3*Delta_x**2*\nu*w_{0}/(2*s_{1}) - Delta_x**2*\nu/s_{1} - Delta_x**2*\nu*w_{0}/(s_{1}*s_{2}) - Delta_x**2*\nu*w_{0}/s_{1}**2 + Delta_x**2*\nu/s_{1}**2

In [12]:
termx=sp.factor(sp.expand(dx*dx*nu*(-w0/4 + sp.Rational(1,6) + w0/(2*s2) + 3*w0/(2*s1) - 1/s1 - w0/(s1*s2) - w0/s1**2 + 1/s1**2 ) ))
termx

-Delta_x**2*\nu*(3*s_{1}**2*s_{2}*w_{0} - 2*s_{1}**2*s_{2} - 6*s_{1}**2*w_{0} - 18*s_{1}*s_{2}*w_{0} + 12*s_{1}*s_{2} + 12*s_{1}*w_{0} + 12*s_{2}*w_{0} - 12*s_{2})/(12*s_{1}**2*s_{2})

In [13]:
termx2=sp.factor(sp.expand(dx*dx*nu*(-w0/4 + sp.Rational(1,6) + w0/(2*s2) + 3*w0/(2*s1) - 1/s1 - w0/(s1*s2) - w0/s1**2 + 1/s1**2 ) ))/(dx*dx*nu)*12*s1*s1*s2
termx2.subs(w0,sp.Rational(2,3))

4*s_{1}**2 - 8*s_{1} + 4*s_{2}

In [15]:
# from sympy import latex
# print(latex(sp.simplify(kept.subs(dt**2*dx**2,0))))